# 2. consumo_privado


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from itertools import product
from sklearn.model_selection import GridSearchCV
import optuna
from optuna.samplers import TPESampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from functools import partial
import joblib
import warnings
import tensorflow as tf
import random

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.get_logger().setLevel('ERROR')
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.set_visible_devices([], 'GPU')
warnings.filterwarnings("ignore")

cwd_path = os.getcwd()
data_path = os.path.join(cwd_path, 'data')
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')
target_var = 'consumo_privado'

## Importación de datos

In [2]:
# función para obtener el trimestre al que pertenece el mes y la fecha
# del último mes del trimestre respectivo
def get_last_day_of_quarter(date):
    quarter = date.quarter
    year = date.year
    if quarter == 1:
        return pd.Timestamp(f'{year}-03-31')
    elif quarter == 2:
        return pd.Timestamp(f'{year}-06-30')
    elif quarter == 3:
        return pd.Timestamp(f'{year}-09-30')
    else:
        return pd.Timestamp(f'{year}-12-31')
# función para sumar si no hay ningún valor nulo o devolver nulo de lo contrario
def sum_or_nan(x):
    return np.nan if x.isna().any() else x.sum()
# función para promediar si no hay ningún valor nulo o devolver nulo de lo contrario
def mean_or_nan(x):
    return np.nan if x.isna().any() else x.mean()

In [3]:
# se lee la metadata
metadata_path = os.path.join(cwd_path,'metadatos.xlsx')
variables = pd.read_excel(metadata_path)
# se agrega el tipo de función de agregación dependiendo de la naturaleza
# de la variable
variables['agg_func'] = np.where(variables.subsector.isin(['tipo_de_cambio', 'tasa_lider', 'tasa_de_interes', 'indice_mensual_actividad_economica', 'inflacion_de_guatemala']),
                                 mean_or_nan, sum_or_nan)

# se toman las filas donde la componente tiene un 1
filtro_componente = (variables[target_var]
                     .fillna(0)
                     .astype(bool))
variables_componente = variables[filtro_componente].reset_index(drop=True)
# se dividen las variables en periodicidades trimestrales y no trimestrales
variables_componente_trimestrales = variables_componente[variables_componente.frecuencia.str.lower() == 'trimestral'].reset_index(drop=True)
variables_componente_no_trimestrales = variables_componente[variables_componente.frecuencia.str.lower() != 'trimestral'].reset_index(drop=True)

# se crea el diccionario para leer los archivos csv de las variables trimestrales
input_files_trimestrales = {}
for idx, row in variables_componente_trimestrales.iterrows():
  input_files_trimestrales[row.variable] = [row.sector, row.subsector, f'{row.variable}.csv']
# se leen las variables
input_list_trimestrales = []
for name, path in input_files_trimestrales.items():
  file_path = os.path.join(input_path, *path)
  df_input = (pd
                .read_csv(file_path, sep=',')
                .rename(columns={'Fecha': 'fa', 'Valor': name})
                .assign(fa = lambda df: pd.to_datetime(df['fa'], dayfirst=True))
                .set_index('fa')
                )
  input_list_trimestrales.append(df_input)
# se crea el dataframe
try:
  df_trimestrales = pd.concat(input_list_trimestrales, axis=1, join='outer')
except ValueError:
  df_trimestrales = pd.DataFrame()

# se crea el diccionario para leer los archivos csv de las variables no trimestrales
input_files_no_trimestrales = {}
for idx, row in variables_componente_no_trimestrales.iterrows():
  input_files_no_trimestrales[row.variable] = [row.sector, row.subsector, f'{row.variable}.csv']
# se leen las variables
input_list_no_trimestrales = []
for name, path in input_files_no_trimestrales.items():
  file_path = os.path.join(input_path, *path)
  df_input = (pd
                .read_csv(file_path, sep=',')
                .rename(columns={'Fecha': 'fa', 'Valor': name})
                .assign(fa = lambda df: pd.to_datetime(df['fa'], dayfirst=True))
                .set_index('fa')
                )
  input_list_no_trimestrales.append(df_input)
# se crea el dataframe
try:
  df_no_trimestrales = pd.concat(input_list_no_trimestrales, axis=1, join='outer')
  # se transforman las variables no trimestrales a trimestrales
  agg_vars = dict()
  for column in df_no_trimestrales.columns:
    agg_vars[column] = variables[variables.variable == column].agg_func.values[0]
  df_no_trimestrales['fa'] = df_no_trimestrales.index
  df_no_trimestrales.fa = df_no_trimestrales.fa.apply(get_last_day_of_quarter)
  df_no_trimestrales.reset_index(drop=True, inplace=True)
  df_no_trimestrales = df_no_trimestrales.groupby('fa').agg(agg_vars)
except ValueError:
  df_no_trimestrales = pd.DataFrame()

# se unen las variables trimestrales con las no trimestrales transformadas
df_list = []
if len(df_trimestrales) > 0:
  df_list.append(df_trimestrales)
if len(df_no_trimestrales) > 0:
  df_list.append(df_no_trimestrales)

df = pd.concat(df_list, axis=1, join='outer')
df.dropna(inplace=True)
output_filepath = os.path.join(output_path, f'{target_var}_dataset.xlsx')
df.to_excel(output_filepath)

In [9]:
input_list_no_trimestrales[0]

,ipc_total
fa,
2009-10-31,-0.65
2009-11-30,-0.61
2009-12-31,-0.28
2010-01-31,1.43
2010-02-28,2.48
...,...
2024-06-30,3.62
2024-07-31,3.78
2024-08-31,3.07


In [4]:
df_trimestrales

,gasto_consumo_final_hogar_isflsh
fa,
2009-06-30,7645.80
2009-09-30,8728.90
2009-12-31,9664.60
2010-03-31,7904.40
2010-06-30,8587.80
...,...
2023-06-30,23129.23
2023-09-30,23189.90
2023-12-31,24929.79


In [5]:
df_no_trimestrales

,ipc_total,tasa_int_lider_pm,oper_estab_mon_me,oper_estab_mon_mn,egreso_div_cap_imp,egreso_div_servicios_trans,egreso_div_transf_donaciones,egreso_div_turismo_viajes,ingreso_div_exp,ingreso_div_servicios_seg,...,taxs,comunic,finan,inmob,act_profes,servicios,ensenanza,salud,otr_act,tipo_cambio_de_referencia_gtq_usd
fa,,,,,,,,,,,,,,,,,,,,,
2009-09-30,NaN,NaN,NaN,NaN,1505896.7,7484.1,10211.9,125283.0,727724.2,7180.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.186359
2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.003248
2010-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.030767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.831462
2024-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.810840
2024-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.770799


In [10]:
df

,gasto_consumo_final_hogar_isflsh,ipc_total,tasa_int_lider_pm,oper_estab_mon_me,oper_estab_mon_mn,egreso_div_cap_imp,egreso_div_servicios_trans,egreso_div_transf_donaciones,egreso_div_turismo_viajes,ingreso_div_exp,...,taxs,comunic,finan,inmob,act_profes,servicios,ensenanza,salud,otr_act,tipo_cambio_de_referencia_gtq_usd
fa,,,,,,,,,,,,,,,,,,,,,


## Procesamiento

In [ ]:
def data_processing(df, target_var, t_samples, keep_all):
    df_shifted = df[[target_var]].copy()
    for n in range(1, t_samples + 1):
        for col in df.columns:
            df_shifted[f'{col}_t_{n}'] = df[col].shift(n)
    df_shifted = df_shifted.dropna()

    for col in df.columns:
        df_shifted[f'{col}_prom_{t_samples}'] = df_shifted[[f'{col}_t_{i}' for i in range(1, t_samples + 1)]].mean(axis=1)
        df_shifted[f'{col}_desv_{t_samples}'] = df_shifted[[f'{col}_t_{i}' for i in range(1, t_samples + 1)]].std(axis=1)
        df_shifted[f'{col}_min_{t_samples}'] = df_shifted[[f'{col}_t_{i}' for i in range(1, t_samples + 1)]].min(axis=1)
        df_shifted[f'{col}_max_{t_samples}'] = df_shifted[[f'{col}_t_{i}' for i in range(1, t_samples + 1)]].max(axis=1)
        df_shifted[f'{col}_sum_{t_samples}'] = df_shifted[[f'{col}_t_{i}' for i in range(1, t_samples + 1)]].sum(axis=1)

    if not keep_all:
        columns_to_keep = [target_var] + [f'{col}_t_{1}' for col in df.columns] + \
                          [f'{col}_{agg}_{t_samples}' for col in df.columns for agg in ['prom', 'desv', 'min', 'max', 'sum']]
        df_shifted = df_shifted[columns_to_keep]

    return df_shifted

df_shifted = data_processing(df, target_var, 4*3, keep_all=False)

X = df_shifted.drop(columns=[target_var])
y = df_shifted[target_var]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(
    X_train_scaled, 
    index=X_train.index,
    columns=X_train.columns
)

X_test_scaled = pd.DataFrame(
    X_test_scaled, 
    index=X_test.index, 
    columns=X_test.columns
)

pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train_pca = pd.DataFrame(
    X_train_pca,
    index=X_train.index,
    columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])]
)

X_test_pca = pd.DataFrame(
    X_test_pca,
    index=X_test.index,
    columns=[f'PC{i+1}' for i in range(X_test_pca.shape[1])]
)

## Entrenamiento

In [ ]:
def optimize_arima(y_train, X_train, p_values, d_values, q_values):
    best_score, best_params, best_model = float("inf"), None, None
    
    for p, d, q in product(p_values, d_values, q_values):
        try:
            model = ARIMA(y_train, order=(p, d, q), exog=X_train).fit()
            if model.aic < best_score:
                best_score, best_params, best_model = model.aic, (p, d, q), model
        except Exception as e:
            continue

    return best_model, best_params

def arquitectura_ann(cant_capas, cant_neurons, learning_rate, input_dim):
    model = Sequential([Input(shape=(input_dim,))])
    for _ in range(cant_capas):
        model.add(Dense(cant_neurons, activation='relu', kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.5))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

def optimizar_ann(trial, X_train, y_train):
    cant_capas = trial.suggest_int('cant_capas', 2, 8, 2)
    cant_neurons = trial.suggest_int('cant_neurons', 8, 64, 8)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    learning_rate = trial.suggest_categorical('learning_rate', [0.001,0.01, 0.1])
    
    model = arquitectura_ann(cant_capas, cant_neurons, learning_rate, X_train.shape[1])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(
        X_train, y_train, 
        validation_split=0.2,
        epochs=100, batch_size=batch_size, verbose=0, callbacks=[early_stopping]
    )
    return min(history.history['val_loss'])

In [ ]:
def train_models(X_train, X_test, y_train, y_test):
    seed_value = 42
    best_models = {}

    p_values = range(1, 4)
    model_ar, best_ar_params = optimize_arima(y_train.asfreq('QE'), X_train.asfreq('QE'), p_values, [0], [0])
    best_models['ar'] = model_ar
    print(f"AR Best Parameters: {best_ar_params}")

    p_values = range(1, 4)
    d_values = range(1, 4)
    q_values = range(1, 4)
    model_arima, best_arima_params = optimize_arima(y_train.asfreq('QE'), X_train.asfreq('QE'), p_values, d_values, q_values)
    best_models['arima'] = model_arima
    print(f"ARIMA Best Parameters: {best_arima_params}")

    params_ridge = {'alpha': [0.1, 1.0, 10],
        'random_state': [seed_value]}
    grid_ridge = GridSearchCV(Ridge(), params_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_ridge.fit(X_train, y_train)
    best_models['lr'] = grid_ridge.best_estimator_
    print(f"LR Ridge Best Parameters: {grid_ridge.best_params_}")
    
    params_knn = {'n_neighbors': [3, 5, 7, 10],
                  'weights': ['uniform', 'distance'],
                  'metric': ['euclidean', 'manhattan', 'minkowski']}
    grid_knn = GridSearchCV(KNeighborsRegressor(),
                            params_knn, cv=3, scoring='neg_mean_squared_error',
                            n_jobs=-1, verbose=0)
    grid_knn.fit(X_train, y_train)
    best_models['knn'] = grid_knn.best_estimator_
    print(f"KNN Best Parameters: {grid_knn.best_params_}")

    params_rf = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [5, 10, 20],
        'max_features': ['auto', 'sqrt'],
        'random_state': [seed_value]
    }
    grid_rf = GridSearchCV(RandomForestRegressor(),
                           params_rf, cv=3, scoring='neg_mean_squared_error',
                           n_jobs=-1, verbose=0)
    grid_rf.fit(X_train, y_train)
    best_models['rf'] = grid_rf.best_estimator_
    print(f"Random Forest Best Parameters: {grid_rf.best_params_}")

    params_xgb = {
        'alpha': [0, 0.1, 1],
        'lambda': [0, 0.1, 1],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 10, 15],
        'random_state': [seed_value]
    }
    grid_xgb = GridSearchCV(xgb.XGBRegressor(n_jobs=-1),
                            params_xgb,
                            cv=3, scoring='neg_mean_squared_error',
                            verbose=0)
    grid_xgb.fit(X_train, y_train)
    best_models['xgb'] = grid_xgb.best_estimator_
    print(f"XGBoost Best Parameters: {grid_xgb.best_params_}")

    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
    study.optimize(
        partial(optimizar_ann, X_train=X_train, y_train=y_train),
        n_trials=30,
        n_jobs=-1,
        show_progress_bar=True)

    best_params = study.best_params
    print(f"ANN Best Parameters: {best_params}")

    model_ann = arquitectura_ann(best_params['cant_capas'], best_params['cant_neurons'], best_params['learning_rate'], X_train.shape[1])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_ann.fit(
        X_train, y_train, 
        validation_split=0.2,
        epochs=100, batch_size=best_params['batch_size'], verbose=0, callbacks=[early_stopping]
    )
    best_models['ann'] = model_ann

    return best_models

best_models = train_models(X_train_pca, X_test_pca, y_train, y_test)

## Evaluación

In [ ]:
def evaluate(best_models, X_train, y_train, X_test, y_test, target_var):
    results = {}
    for name, model in best_models.items():
        # Predicción y cálculo del MSE en el conjunto de entrenamiento
        if name in ('ar', 'arima'):
            y_train_pred = model.predict(start=0, end=len(y_train)-1, exog=X_train)
        else:
            y_train_pred = model.predict(X_train)
        train_mse = mean_squared_error(y_train, y_train_pred)
        
        # Predicción y cálculo del MSE en el conjunto de prueba
        if name in ('ar', 'arima'):
            y_test_pred = model.predict(start=len(y_train), end=len(y_train)+len(y_test)-1, exog=X_test)
        else:
            y_test_pred = model.predict(X_test)
        test_mse = mean_squared_error(y_test, y_test_pred)

        train_rmse = np.sqrt(train_mse)
        test_rmse = np.sqrt(test_mse)
        
        # Almacenar los resultados
        results[name] = {
            'y_train_pred': y_train_pred,
            'train_rmse': train_rmse,
            'y_test_pred': y_test_pred,
            'test_rmse': test_rmse
        }
        
        print(f"{name}: train_rmse={train_rmse:,.2f}, test_rmse={test_rmse:,.2f}")

    best_model_name = min(results, key=lambda x: results[x]['test_rmse'])
    best_model = best_models[best_model_name]
    print(f"Mejor modelo: {best_model_name}")

    export_path = os.path.join(output_path, 'models')
    with open(os.path.join(export_path ,f"{target_var}.pkl"), "wb") as f:
        joblib.dump(best_model, f)

    print(f"Modelo {target_var} exportado exitosamente.")
    return results

results = evaluate(best_models, X_train_pca, y_train, X_test_pca, y_test, target_var)

In [ ]:
def plot_comparison(results, target_var):
    colors = {'ar' : 'brown',
              'arima' : 'blue',
              'lr' : 'red',
              'knn' : 'orange',
              'rf' : 'green',
              'xgb' : 'purple',
              'ann' : 'pink'}
    
    plt.figure(figsize=(8, 4))
    plt.plot(y_test.values, label="Valores Reales (y_test)", color="blue")
    
    for name, result in results.items():
        if name in ('ar', 'arima'):
            plt.plot(result['y_test_pred'].reset_index(drop=True), label=f"y_test_pred_{name}", color=colors[name], linestyle="--")
        else:
            plt.plot(result['y_test_pred'], label=f"y_test_pred_{name}", color=colors[name], linestyle="--")

    plt.xlabel("tiempo")
    plt.ylabel(target_var)
    plt.title(f"Resultados {target_var}")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_path ,f'result_{target_var}.png'), dpi=300)
    plt.show()

plot_comparison(results, target_var)